In [1]:
import os
import sys
import json
if 'SUMO_HOME' in os.environ:
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
import sumolib
import importlib
import LLAMAconnect

In [30]:
# Latitude e longitude de locais de interesse
coords = {
    "IC": (-22.813344, -47.063667),
    "FEEC": (-22.820788, -47.066489),
    "HOME": (-22.823781, -47.074111),
    "RU": (-22.817327, -47.071338)
}

In [31]:
net = sumolib.net.readNet('osm.net.xml')
radius = 20
parkingAreas = list(sumolib.output.parse('osm_stops.add.xml', "parkingArea"))

In [32]:
def has_parking_spot(lanes, parkingAreas):
    # Example of parkingArea:
    # <parkingArea id="pa-1046248579#0" lane="-1046248579#0_0" roadsideCapacity="94" length="5.00"/>
    # Returns parkingArea id if there is a parking spot in the lane
    lane_ids = [lane.getID() for lane in lanes]
    for park in parkingAreas:
        if park.lane in lane_ids:
            return park.id
    return False

In [33]:
def getClosestEdges(lat, lon, radius):
    # Gets the 10 closest edges to the given lat, lon
    x, y = net.convertLonLat2XY(lon, lat)
    edges = net.getNeighboringEdges(x, y, radius)
    if (len(edges) > 0):
        distanceAndEdges = sorted([(dist, edge) for edge, dist in edges], key=lambda x:x[0])
        closestEdges = [x[1] for x in distanceAndEdges[:10]]
    
    return closestEdges

In [41]:
def getParkingSpot(lat, lon, radius, parkingAreas):
    # Get the parking spot closest to the given lat, lon
    # Used to set stops for the vehicles

    edges = getClosestEdges(lat, lon, radius)
    # Look for parking spots
    for i in range(len(edges)):
        parking_spot = has_parking_spot(edges[i].getLanes(), parkingAreas)
        if parking_spot:
            return parking_spot

    print("No parking spot found")
    return None

In [35]:
def getPath(location_time_list, parkingAreas):
    # All that is needed to create the trip are the stops (parking areas) and the start and end edges.
    # The duarouter is responsible for finding the path between the edges going through the stops.
    # Here, we get the edges and stops that are going to be sent to LLAMA to create the trip.

    # 'coordinates' is a list of tuples with the latitude and longitude of the points of interest, for example IC, FEEC, IC means that
    # the vehicle will start from IC, stop at a parking lot close to FEEC, and then back to IC.
    # The first and last coordinates should be edges and the others should be parking spots.

    # Departure for 7 is 0, 8 is 100, 9 is 200 and so on
    stop_durations = []
    departures = list(location_time_list.keys())
    stop_durations.append(-1) # Indicates this is an edge and not a parking spot

    path = []
    locations = list(location_time_list.values())
    home = getClosestEdges(*locations[0], radius)[0].getID()
    path.append(home) 
    
    for i in range(1, len(locations)-1):
        stop_durations.append(100 * (departures[i] - departures[i - 1]))
        path.append(getParkingSpot(*locations[i], radius, parkingAreas))
    
    path.append(home)
    stop_durations.append(-1)
    
    return path, stop_durations

In [36]:
def pathToXML(path, vehicleID, departure_time, stop_durations):
    # Converts the path to the XML format that LLAMA understands
    xml = f'<trip id="{vehicleID}" depart="{departure_time}" from="{path[0]}" to="{path[-1]}">\n'
    for i in range(1, len(path)-1):
        xml += f'\t<stop parkingArea="{path[i]}" duration="{stop_durations[i]}"/>\n'
    xml += '</trip>'
    return xml

Getting the routine for the student described

In [37]:
student_info = "Computer Engineering student. He has most classes at FEEC and IC and has luch at RU and dinner at RU. The student is never late to classes. The student lives close to the university and likes to relax at home whenever possible. The student starts and ends his day at home"
importlib.reload(LLAMAconnect)
response = LLAMAconnect.getResponse_trip(student_info)
json_response = json.loads(response)
print(response)

{
   "7": {"location": "HOME", "activity": "wake up"},
   "8": {"location": "FEEC", "activity": "class"},
   "9": {"location": "FEEC", "activity": "class"},
   "10": {"location": "FEEC", "activity": "class"},
   "11": {"location": "FEEC", "activity": "class"},
   "12": {"location": "RU", "activity": "lunch"},
   "13": {"location": "FEEC", "activity": "class"},
   "14": {"location": "IC", "activity": "class"},
   "15": {"location": "IC", "activity": "class"},
   "16": {"location": "IC", "activity": "class"},
   "17": {"location": "IC", "activity": "class"},
   "18": {"location": "RU", "activity": "dinner"},
   "19": {"location": "HOME", "activity": "relax"},
   "20": {"location": "HOME", "activity": "relax"}
}


Getting the latitude and longitude for each of the locations

TO DO - If a trip or flow defines at least one stop as child element, the attributes 'from' and 'to' may be omitted. 

In [45]:
location_time_list= {}
last = coords[json_response[f'{7}']['location']]
location_time_list[7] = last
for i in range(1, len(json_response)):
    local = coords[json_response[f'{i + 7}']['location']]
    if local != last:
        location_time_list[i + 7] = local
        last = local

print(location_time_list)

{7: (-22.823781, -47.074111), 8: (-22.820788, -47.066489), 12: (-22.817327, -47.071338), 13: (-22.820788, -47.066489), 14: (-22.813344, -47.063667), 18: (-22.817327, -47.071338), 19: (-22.823781, -47.074111)}


Taking the locations and turning them into a XML trip

In [46]:
path, stop_durations = getPath(location_time_list, parkingAreas)
xml = pathToXML(path, "vehicle1", 0, stop_durations)
print(path)
print(xml)

['563505734#2', 'pa693637406#7', 'pa72508815#14', 'pa693637406#7', 'pa75701374#0', 'pa72508815#14', '563505734#2']
<trip id="vehicle1" depart="0" from="563505734#2" to="563505734#2">
	<stop parkingArea="pa693637406#7" duration="100"/>
	<stop parkingArea="pa72508815#14" duration="400"/>
	<stop parkingArea="pa693637406#7" duration="100"/>
	<stop parkingArea="pa75701374#0" duration="100"/>
	<stop parkingArea="pa72508815#14" duration="400"/>
</trip>


In [52]:
# Trying to get response from LLAMA, but it's not working properly
response = LLAMAconnect.getResponse_xml(path, "veh_1", 10, 100)
print(response)

```
{
  "trip": {
    "id": "veh_1",
    "type": "veh_passenger",
    "depart": "10",
    "from": "FEEC",
    "to": "FEEC",
    "stop": [
      {
        "parkingArea": "pa75701374#0",
        "duration": "100"
      }
    ]
  }
}
```
